In [5]:
pip install pandas numpy matplotlib prophet scikit-learn streamlit fredapi yfinance


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
# 🔧 Manipulación y visualización
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 🔮 Modelado
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error

# 🌐 APIs externas
import yfinance as yf
from fredapi import Fred

# 📊 Streamlit (para la app final)
import streamlit as st


In [9]:
# 🔑 Inicializar FRED
from fredapi import Fred
fred = Fred(api_key='437ffc22620f0fe3615350b1764f112b')

# 📥 Descargar series desde FRED
inflacion_raw = fred.get_series('CPIAUCSL')
tasa_raw = fred.get_series('FEDFUNDS')

# 🧼 Convertir a DataFrame con fechas
inflacion_mejorado = pd.DataFrame({
    'Fecha': inflacion_raw.index,
    'Inflacion': inflacion_raw.values
})
tasa_mejorado = pd.DataFrame({
    'Fecha': tasa_raw.index,
    'Tasa_Fed': tasa_raw.values
})

# 📅 Filtrar rango deseado
inflacion_mejorado = inflacion_mejorado[(inflacion_mejorado['Fecha'] >= '2010-01-01') & (inflacion_mejorado['Fecha'] <= '2025-12-31')]
tasa_mejorado = tasa_mejorado[(tasa_mejorado['Fecha'] >= '2010-01-01') & (tasa_mejorado['Fecha'] <= '2025-12-31')]


In [13]:
# 📥 Descargar series desde Yahoo Finance
usdeur_raw = yf.download('EURUSD=X', start='2010-01-01', end='2025-12-31')
dxy_raw = yf.download('DX-Y.NYB', start='2010-01-01', end='2025-12-31')

# 🧼 Limpiar y renombrar
usdeur_mejorado = usdeur_raw[['Close']].reset_index().rename(columns={'Date': 'Fecha', 'Close': 'USD_EUR'})
dxy_mejorado = dxy_raw[['Close']].reset_index().rename(columns={'Date': 'Fecha', 'Close': 'DXY'})

# 🧼 Eliminar MultiIndex si existe
usdeur_mejorado.columns = usdeur_mejorado.columns.get_level_values(0)
dxy_mejorado.columns = dxy_mejorado.columns.get_level_values(0)

# 📅 Convertir fechas a mensual y ordenar
usdeur_mejorado = usdeur_mejorado.set_index('Fecha').resample('MS').mean().reset_index().sort_values('Fecha')
dxy_mejorado = dxy_mejorado.set_index('Fecha').resample('MS').mean().reset_index().sort_values('Fecha')
inflacion_mejorado = inflacion_mejorado.sort_values('Fecha')
tasa_mejorado = tasa_mejorado.sort_values('Fecha')

# 🧩 Unir todas las series
df_mejorado = usdeur_mejorado.merge(dxy_mejorado, on='Fecha', how='inner')
df_mejorado = df_mejorado.merge(inflacion_mejorado, on='Fecha', how='inner')
df_mejorado = df_mejorado.merge(tasa_mejorado, on='Fecha', how='inner')

# 🧽 Eliminar nulos y guardar
df_mejorado = df_mejorado.dropna()
df_mejorado.to_csv('dataset_mejorado.csv', index=False)


C:\Users\cpuertas\AppData\Local\Temp\ipykernel_16364\2935753380.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  usdeur_raw = yf.download('EURUSD=X', start='2010-01-01', end='2025-12-31')
[*********************100%***********************]  1 of 1 completed
C:\Users\cpuertas\AppData\Local\Temp\ipykernel_16364\2935753380.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  dxy_raw = yf.download('DX-Y.NYB', start='2010-01-01', end='2025-12-31')
[*********************100%***********************]  1 of 1 completed


In [15]:
# 📥 Cargar dataset
df = pd.read_csv('dataset_mejorado.csv')

# 🧼 Renombrar columnas para Prophet
df = df.rename(columns={
    'Fecha': 'ds',
    'USD_EUR': 'y',
    'DXY': 'dxy_mejorado',
    'Inflacion': 'inflacion_mejorado',
    'Tasa_Fed': 'tasa_mejorado'
})

# 🧽 Convertir tipos
df['ds'] = pd.to_datetime(df['ds'])
df = df.dropna()

# 🔮 Entrenar modelo Prophet con regresores
modelo = Prophet(yearly_seasonality=False)
modelo.add_regressor('dxy_mejorado')
modelo.add_regressor('inflacion_mejorado')
modelo.add_regressor('tasa_mejorado')
modelo.fit(df)

# 📈 Crear proyección futura (36 meses)
futuro = modelo.make_future_dataframe(periods=36, freq='MS')
ultimo = df.iloc[-1]

# Trayectorias suavizadas
futuro['dxy_mejorado'] = np.linspace(ultimo['dxy_mejorado'], ultimo['dxy_mejorado'] * 0.98, len(futuro))
futuro['inflacion_mejorado'] = np.linspace(ultimo['inflacion_mejorado'], ultimo['inflacion_mejorado'] * 0.97, len(futuro))
futuro['tasa_mejorado'] = np.linspace(ultimo['tasa_mejorado'], ultimo['tasa_mejorado'] * 0.96, len(futuro))

# 🔮 Generar predicción
forecast = modelo.predict(futuro)

# 📊 Validación del modelo (solo sobre datos históricos)
real = df['y'].reset_index(drop=True)
pred = forecast.loc[:len(real)-1, 'yhat'].reset_index(drop=True)

mae = mean_absolute_error(real, pred)
rmse = mean_squared_error(real, pred) ** 0.5


print(f"✅ MAE Prophet mejorado: {mae:.4f}")
print(f"✅ RMSE Prophet mejorado: {rmse:.4f}")

# 💾 Guardar modelo
import pickle
with open('modelo_prophet_mejorado.pkl', 'wb') as f:
    pickle.dump(modelo, f)


11:56:33 - cmdstanpy - INFO - Chain [1] start processing
11:56:33 - cmdstanpy - INFO - Chain [1] done processing


✅ MAE Prophet mejorado: 0.1063
✅ RMSE Prophet mejorado: 0.1201


In [16]:
import pandas as pd
import yfinance as yf
from fredapi import Fred
from datetime import datetime

# Clave API de FRED
fred = Fred(api_key="437ffc22620f0fe3615350b1764f112b")

# 1. Descargar USD/EUR y DXY desde Yahoo Finance
eurusd = yf.download("EURUSD=X", start="2010-01-01", end=datetime.today().strftime('%Y-%m-%d'))
dxy = yf.download("DX-Y.NYB", start="2010-01-01", end=datetime.today().strftime('%Y-%m-%d'))

# 2. Descargar inflación y tasa de interés desde FRED
inflacion = fred.get_series("CPIAUCSL")
tasa_fed = fred.get_series("FEDFUNDS")

# 3. Formatear y unir los datos
df = pd.DataFrame()
df["Fecha"] = eurusd.index
df["USD_EUR"] = eurusd["Close"].values
df["DXY"] = dxy["Close"].reindex(eurusd.index).values
df["Inflacion"] = inflacion.reindex(eurusd.index.to_period("M").to_timestamp()).values
df["Tasa_Fed"] = tasa_fed.reindex(eurusd.index.to_period("M").to_timestamp()).values

# 4. Guardar el dataset actualizado
df.dropna(inplace=True)
df.to_csv("dataset_final_economico.csv", index=False)


C:\Users\cpuertas\AppData\Local\Temp\ipykernel_16364\4016925914.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  eurusd = yf.download("EURUSD=X", start="2010-01-01", end=datetime.today().strftime('%Y-%m-%d'))
[*********************100%***********************]  1 of 1 completed
C:\Users\cpuertas\AppData\Local\Temp\ipykernel_16364\4016925914.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  dxy = yf.download("DX-Y.NYB", start="2010-01-01", end=datetime.today().strftime('%Y-%m-%d'))
[*********************100%***********************]  1 of 1 completed


In [17]:
from prophet import Prophet
import pandas as pd
import numpy as np

# Cargar el dataset actualizado
df = pd.read_csv("dataset_final_economico.csv")

# Preparar los datos para Prophet
df_prophet = df[["Fecha", "USD_EUR"]].rename(columns={"Fecha": "ds", "USD_EUR": "y"})

# Entrenar el modelo
modelo = Prophet()
modelo.fit(df_prophet)

# Crear horizonte de predicción (3 años)
future = modelo.make_future_dataframe(periods=36, freq="M")
forecast = modelo.predict(future)

# Escenarios
forecast["escenario_neutro"] = forecast["yhat"]
forecast["escenario_positivo"] = forecast["yhat"] * 1.05  # crecimiento del 5%
forecast["escenario_negativo"] = forecast["yhat"] * 0.95  # caída del 5%

# Guardar resultados
forecast_final = forecast[["ds", "escenario_neutro", "escenario_positivo", "escenario_negativo"]]
forecast_final.to_csv("forecast_mejorado.csv", index=False)


12:19:02 - cmdstanpy - INFO - Chain [1] start processing
12:19:03 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\cpuertas\AppData\Roaming\Python\Python313\site-packages\prophet\forecaster.py:1872: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(


In [2]:
pip install plotly


  Using cached plotly-6.2.0-py3-none-any.whl.metadata (8.5 kB)
Using cached plotly-6.2.0-py3-none-any.whl (9.6 MB)
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [21]:
# 📅 Generar fechas futuras (ej. 180 días desde el último dato)
future = modelo_prophet.make_future_dataframe(periods=180)

# 🔮 Generar predicciones
forecast = modelo_prophet.predict(future)

# ✅ Guardar el forecast si quieres reutilizarlo
forecast.to_csv("forecast_mejorado.csv", index=False)

# 🧾 Confirmar columnas disponibles
st.write("Columnas generadas por Prophet:")
st.write(forecast.columns.tolist())


NameError: name 'modelo_prophet' is not defined

In [1]:
pip install streamlit


  Using cached altair-5.5.0-py3-none-any.whl.metadata (11 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached cachetools-6.1.0-py3-none-any.whl.metadata (5.4 kB)
  Using cached click-8.2.1-py3-none-any.whl.metadata (2.5 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl.metadata (7.1 kB)
  Using cached watchdog-6.0.0-py3-none-win_amd64.whl.metadata (44 kB)
  Using cached pydeck-0.9.1-py2.py3-none-any.whl.metadata (4.1 kB)
  Using cached gitdb-4.0.12-py3-none-any.whl.metadata (1.2 kB)
  Using cached smmap-5.0.2-py3-none-any.whl.metadata (4.3 kB)
   ---------------------------------------- 0.0/9.9 MB ? eta -:--:--
   -------- ------------------------------- 2.1/9.9 MB 11.8 MB/s eta 0:00:01
   ------------------- -------------------- 4.7/9.9 MB 11.9 MB/s eta 0:00:01
   ---------------------------- ----------- 7.1/9.9 MB 11.8 MB/s eta 0:00:01
   -------------------------------------- - 9.4/9.9 

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [4]:
pip install -r requirements.txt


  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached curl_cffi-0.12.0-cp39-abi3-win_amd64.whl.metadata (15 kB)
  Using cached GitPython-3.1.44-py3-none-any.whl.metadata (13 kB)
  Using cached holidays-0.76-py3-none-any.whl.metadata (44 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached jsonschema-4.24.1-py3-none-any.whl.metadata (7.5 kB)
  Using cached jsonschema_specifications-2025.4.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached multitasking-0.0.11-py3-none-any.whl.metadata (5.5 kB)
  Using cached narwhals-1.47.1-py3-none-any.whl.metadata (11 kB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Ignored the following versions that require a different python version: 2.3.0 Requires-Python >=3.11; 2.3.1 Requires-Python >=3.11; 2.3.2 Requires-Python >=3.11
ERROR: Could not find a version that satisfies the requirement numpy==2.3.1 (from versions: 1.3.0, 1.4.1, 1.5.0, 1.5.1, 1.6.0, 1.6.1, 1.6.2, 1.7.0, 1.7.1, 1.7.2, 1.8.0, 1.8.1, 1.8.2, 1.9.0, 1.9.1, 1.9.2, 1.9.3, 1.10.0.post2, 1.10.1, 1.10.2, 1.10.4, 1.11.0, 1.11.1, 1.11.2, 1.11.3, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 1.13.3, 1.14.0, 1.14.1, 1.14.2, 1.14.3, 1.14.4, 1.14.5, 1.14.6, 1.15.0, 1.15.1, 1.15.2, 1.15.3, 1.15.4, 1.16.0, 1.16.1, 1.16.2, 1.16.3, 1.16.4, 1.16.5, 1.16.6, 1.17.0, 1.17.1, 1.17.2, 1.17.3, 1.17.4, 1.17.5, 1.18.0, 1.18.1, 1.18.2, 1.18.3, 1.18.4, 1.18.5, 1.19.0, 1.19.1, 1.19.2, 1.19.3, 1.19.4, 1.19.5, 1.20.0, 1.20.1, 1.20.2, 1.20.3, 1.21.0, 1.21.1, 1.21.2, 1.21.3, 1.21.4, 1.21.5, 1.21.6, 1.22.0, 1.22.1, 1.22.2, 1.22.3, 1.22.4, 1.23.0, 1.23.1, 1.23.2, 1.23.3, 1.23.4, 1.23.5, 1.24.0, 1.24.1, 1.24.2, 1.24.3, 1.24.4,